CODE for deploying saved NER model


In [5]:
## TODO : make modification on the parse_query part to include "near concept" in the search 
## 
##

In [14]:
pip install -U pip setuptools wheel

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install spacy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install gensim

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
from gensim.parsing.preprocessing import remove_stopwords

text = "Nick likes to play football, however he is not too fond of tennis."
filtered_sentence = remove_stopwords(text)

print(filtered_sentence)

Nick likes play football, fond tennis.


# define model for production
from spacy.lang.en import English
from gensim.parsing.preprocessing import remove_stopwords

class QueryParse ():

  def __init__(self):
    self.nlp_model = English()
    ruler = self.nlp_model.add_pipe("entity_ruler").from_disk("./test1_NER_model_may28.jsonl")

    
  def parse_query_1 (self, qry_request, context): 
    """
    parse user query
    """
    # eventually use lower case - now mixed case
    #doc = self.nlp_model(str.lower(qry_request))
    doc = self.nlp_model(qry_request)
    phrase_list = []
    label_list = []
    comb_list = []
    for ent in doc.ents:
      phrase_list.append(ent.text)
      label_list.append(ent.label_)
      comb_list.append([ent.label_, ent.text])
    
    weaviate_context=qry_request
    for entity in comb_list:
        weaviate_context=weaviate_context.replace(entity[1], '') 
        
    weaviate_context=remove_stopwords(weaviate_context)    
    comb_list.append(["weaviate_neartext", weaviate_context])
    
    if len(label_list) == 1:
      return comb_list[0]
    
    return comb_list



In [266]:
# define model for production
from spacy.lang.en import English
from gensim.parsing.preprocessing import remove_stopwords

class QueryParse ():

  def __init__(self):
    self.nlp_model = English()
    ruler = self.nlp_model.add_pipe("entity_ruler").from_disk("./test1_NER_model_may28.jsonl")

# this version of parse_query returns a dictionary 
    
  def parse_query (self, qry_request, context): 
    """
    parse user query
    """
    # eventually use lower case - now mixed case
    #doc = self.nlp_model(str.lower(qry_request))
    doc = self.nlp_model(qry_request)
    phrase_list = []
    label_list = []
    comb_list = {}
    for ent in doc.ents:
      phrase_list.append(ent.text)
      label_list.append(ent.label_)
      comb_list[ent.label_]= ent.text
    
    # add weaviate_context : just information excluding words for ent.label
    weaviate_context=qry_request
    keys=comb_list.keys()
    for key in keys:
        if key!='weaviate_neartext':
            weaviate_context=weaviate_context.replace(comb_list[key], '')  
    
    weaviate_context=remove_stopwords(weaviate_context)    
    comb_list["weaviate_neartext"]= weaviate_context
    
    #if len(label_list) == 1:
    #  return comb_list[0]
    
    return comb_list


In [268]:
# xx is a dummy variable - we dont have a context etc
# instiate the model 
ner_service = QueryParse ( )
ner_out1 = ner_service.parse_query ("Marco EyeCare Cards from Shoulder Bags and Mitaki-Japan and large size and come highly recomended by friend", 'xx')
print (ner_out1)

{'PRODUCT': 'Marco EyeCare Cards', 'CATEGORY': 'Shoulder Bags', 'weaviate_neartext': 'Mitaki-Japan large size come highly recomended friend'}


In [209]:
# define some text for testing
sample_query_text = "I want to buy a Sunydeal Hedgren Portable Speakers &amp; Docks"

sample_text2 = "compare P46 Digital Pikachu USB Flash Drive 16GB with Sunydeal Portable Speakers &amp; Docks"

text1 = "Marco EyeCare Cards from Shoulder Bags and Mitaki-Japan and large size"

text2 = "Electronic made by Maxell good with deep bass performance"

In [205]:
# we have already instatiated the model - so just keep using pred service
# xx is dummy variable, place holder for context etc
print (ner_service.parse_query (text1, 'xx'))

{'PRODUCT': 'Marco EyeCare Cards', 'CATEGORY': 'Shoulder Bags', 'weaviate_neartext': 'Marco EyeCare Cards Shoulder Bags Mitaki-Japan large size'}


In [235]:
text2_ner=ner_service.parse_query (text2, 'xx')
print ( text2_ner)


{'VENDOR': 'Maxell', 'weaviate_neartext': 'Electronic good deep bass performance'}


In [236]:
text3 = "Laptop Case Moleskine good with water-resistant "


In [237]:
text3_ner=ner_service.parse_query (text3, 'xx') 
print(text3_ner)

{'VENDOR': 'Moleskine', 'weaviate_neartext': 'Laptop Case good water-resistant'}


## get context search part 

In [238]:
# initialize weaviate client
import weaviate

# for sandbox 
#weaviate_url = ' https://mlopssearchtest.semi.network'
#secret = weaviate.AuthClientPassword("sridiyer@gmail.com", "xxx")

weaviate_url = 'http://34.67.249.252:8080/'
secret = weaviate.AuthClientPassword("admin", "admin")

# Initiate the client with the secret
client = weaviate.Client(weaviate_url, secret)

In [342]:
def get_query_for_ner_concept(text_ner):
    
    keys=text_ner.keys()
    operands_list=[]
    for key in keys:
        if key!='weaviate_neartext':
            if key =='VENDOR':
                operands_list.append({
                     "path": ["brand"],
                     "operator": "Equal",
                     "valueString": text_ner['VENDOR']
                   })
            elif key =='CATEGORY':
                 operands_list.append({
                     "path": ["category"],
                     "operator": "Equal",
                     "valueString": text_ner['CATEGORY']
                   })
            else: 
                continue 
                
    where_filter = {
          "operator": "Or",
          "operands": operands_list
        }    
    near_text_filter = {
            "concepts": [text_ner['weaviate_neartext']]
           # "certainty": 0.7
        }
    query_result = client.query\
            .get("Product", ["title","category", "brand" ,"description","price","mainCategory"])\
            .with_near_text(near_text_filter)\
            .with_limit(5)\
            .with_where(where_filter)\
            .do()
       #         .with_near_text(near_text_filter)\

    return(query_result['data']['Get']['Product'])


In [346]:
get_query_for_ner_concept(text3_ner)

[{'brand': 'Moleskine',
  'category': ['Electronics',
   'Computers &amp; Accessories',
   'Laptop Accessories',
   'Bags, Cases &amp; Sleeves',
   'Folio Cases'],
  'description': 'Designed to embrace and safeguard your digital creativity, Moleskine laptop cases are fitted with stretch fabric, velour lining and light padding to ensure a high level of protection.',
  'mainCategory': 'Office Products',
  'price': 45.74,
  'title': 'Moleskine 9788866139799 15-Inch Laptop Case'},
 {'brand': 'Moleskine',
  'category': ['Electronics',
   'Computers & Accessories',
   'Tablet Accessories',
   'Bags, Cases & Sleeves'],
  'description': 'Designed to embrace and safeguard your digital creativity, Moleskine laptop cases are fitted with stretch fabric, velour lining and light padding to ensure a high level of protection.',
  'mainCategory': 'Computers',
  'price': 37.52,
  'title': 'Moleskine 9788866139812 10-Inch Laptop Case'},
 {'brand': 'Moleskine',
  'category': ['Electronics', 'eBook Readers

KeyError: 'VENDOR'

In [336]:
text3_ner

{'VENDOR': 'Moleskine',
 'weaviate_neartext': 'Laptop Case good water-resistant'}

In [339]:
ner_out1

{'PRODUCT': 'Marco EyeCare Cards',
 'CATEGORY': 'Shoulder Bags',
 'weaviate_neartext': 'Mitaki-Japan large size come highly recomended friend'}

In [344]:
text_ner=text3_ner
keys=text_ner.keys()
operands_list=[]
for key in keys:
    if key!='weaviate_neartext':
        if key =='VENDOR':
            operands_list.append({
                 "path": ["brand"],
                 "operator": "Equal",
                 "valueString": text_ner['VENDOR']
               })
        elif key =='CATEGORY':
             operands_list.append({
                 "path": ["category"],
                 "operator": "Equal",
                 "valueString": text_ner['CATEGORY']
               })
        else: 
            continue 

In [345]:
operands_list

[{'path': ['brand'], 'operator': 'Equal', 'valueString': 'Moleskine'}]